# Let's play with our model: making predictions

In [3]:
from transformers import AutoModelForSequenceClassification, AutoConfig
import yaml

config = yaml.safe_load(open("project/config.yml", "r"))
training_config = config["training_config"]
model_name = training_config["model_name"]
num_labels = training_config["num_labels"]

config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## On charge les poids

In [4]:
model = model.from_pretrained("project/results/checkpoint-21400", num_labels=num_labels)

## Essayons une prédiction sur deux phrases

In [5]:
dic = {0: "entailment", 1: "neutral", 2: "contradiction"}
#Exemple d'input

input1 = "A man eats a pizza."
input2 = "The man is sleeping"

In [6]:
from transformers import AutoTokenizer
import numpy as np
import torch

dic = {0: "entailment", 1: "neutral", 2: "contradiction"}

def prediction(input1, input2):
    #On tokénize avec nos inputs avec notre modèle
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    inputs = tokenizer(input1, input2, return_tensors="pt")
    outputs = model(**inputs)

    #On réalise la prédiction
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    label = np.argmax(predictions.detach().numpy())

    return dic[label]


print("C'est deux phrases sont en", prediction(input1, input2))

C'est deux phrases sont en contradiction
